In [1]:
import numpy as np

rawData = np.array([[29,37,39,70,74,84,38,36,26],
                    [27,100,49,72,85,89,83,28,24],
                    [25,47,50,73,86,90,71,82,22],
                    [23,75,79,48,69,87,88,81,18],
                    [19,76,80,78,46,68,67,40,16],
                    [17,41,77,45,35,20,21,32,15],
                    [13,60,43,44,34,33,31,30,14],
                    [12,11,10,9,8,7,6,5,0]])
print(rawData)

[[ 29  37  39  70  74  84  38  36  26]
 [ 27 100  49  72  85  89  83  28  24]
 [ 25  47  50  73  86  90  71  82  22]
 [ 23  75  79  48  69  87  88  81  18]
 [ 19  76  80  78  46  68  67  40  16]
 [ 17  41  77  45  35  20  21  32  15]
 [ 13  60  43  44  34  33  31  30  14]
 [ 12  11  10   9   8   7   6   5   0]]


In [2]:
m = 8
n = 9
vertexList = np.arange(m*n*3, dtype='f')
vertexList = vertexList.reshape(8,9,3)

x = 0
y = 1
z = 2

zScale = 0.1 * 0.5

for i in range(m):
    for j in range(n):
        vertexList[i,j,x] = i
        vertexList[i,j,y] = j
        vertexList[i,j,z] = rawData[i,j] * zScale
# print(vertexList)

In [3]:
#lines
horLineNum = m*(n-1)
verLineNum = n*(m-1)
diaLineNum = (m-1)*(n-1)
totLineNum = 3*m*n - 2*m - 2*n + 1
# print(horLineNum + verLineNum + diaLineNum == TotLineNum)
lineList = np.arange(totLineNum)

# horizontal line
horLine = []
for i in range(m):
    line = np.arange(i*n+1, (i+1)*n+1)
    horLine.append(line)

# vertical line
verLine = []
for j in range(n):
    line = np.arange(j+1, m*n+j+1, n)
    verLine.append(line)
    
# diagonal line
diaLine = []
for i in range(m-1):
    for j in range(n-1):
        line = np.arange(i*n+j+1, (i+2)*n+j+1,n+1)
        diaLine.append(line)

In [4]:
# faces
faceList = []
for i in range(m-1):
    for j in range(n-1):
        face1 = np.array([i*n+j+1, i*n+j+n+1, i*n+j+n+1+1])
        faceList.append(face1)
        face1 = np.array([i*n+j+1, i*n+j+n+1+1, i*n+j+1+1])
        faceList.append(face1)

In [5]:
import datetime

outputFile = open("mesh/carr.obj","w")
now = datetime.datetime.now()
outputFile.write("# Create by Jiahui Liu, " + str(now) + "\n")
outputFile.write("# A 2-D mesh.\n")
outputFile.write("\n")

outputFile.write("# vertices\n")
for i in range(m):
    for j in range(n):
        outputFile.write("v " + str(vertexList[i,j,x]) + " " + str(vertexList[i,j,y]) + " " + str(vertexList[i,j,z]) + "\n")

outputFile.write("\n")
outputFile.write("# faces\n")
for face in faceList:
    outputFile.write("f " + str(face).strip("]").strip("[").strip() + "\n")
        
# outputFile.write("\n")
# outputFile.write("# lines\n")
# for line in horLine:
#     outputFile.write("l " + str(line).strip("]").strip("[").strip() + "\n")
# for line in verLine:
#     outputFile.write("l " + str(line).strip("]").strip("[").strip() + "\n")
# for line in diaLine:
#     outputFile.write("l " + str(line).strip("]").strip("[").strip() + "\n")
    
outputFile.close()

In [10]:
# save in volume, the .dat and .raw file
volumeFile = "volumes/carr"

dat = """RawFile: {file}
Resolution: {dimX} {dimY} {dimZ}
Format: {format}
DataRange: {dataMin} {dataMax}
ValueRange: {dataMin} {dataMax}
BasisVector1: {V1} 0.0 0.0
BasisVector2: 0.0 {V2} 0.0
BasisVector3: 0.0 0.0 {V3}
"""

# write .dat file
with open(volumeFile + ".dat", 'w') as f:
    f.write(dat.format(
        file = volumeFile.split("/")[-1]+".raw", 
        format = "FLOAT32", 
        dimX = rawData.shape[0],
        dimY = rawData.shape[1],
        dimZ = 1,
        dataMin = rawData.min(),
        dataMax = rawData.max(),
        V1 = rawData.shape[0],
        V2 = rawData.shape[1],
        V3 = 1
    ))

arr = np.float32(rawData.T)
arr.tofile(volumeFile + ".raw")
